In [2]:
!pip install "numpy<2" --force-reinstall


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl (15.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


DEPRECATION: Loading egg at c:\program files\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pyjpegls 1.5.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pylibjpeg-libjpeg 2.3.0 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pylibjpeg-openjpeg 2.5.0 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
qiskit-ibmq-provider 0.20.2 requires numpy<1.24, but you

In [3]:
import os
os.listdir('.')   # verify patch_extraction, read_dicom_wsi, extract_sqlite... are visible

from patch_extraction import extract_patches_with_classes

In [3]:
db_path = r"D:\Digital_Pathology_Edge\data\raw\train\MITOS_WSI_CCMCT_ODAEL_train_dcm.sqlite"
slides_dir = r"D:/Digital_Pathology_Edge/data/raw/train/"
output_dir = r"D:/Digital_Pathology_Edge/data/patches/train/"

extract_patches_with_classes(db_path, slides_dir, output_dir)


Error loading D:/Digital_Pathology_Edge/data/raw/train/c91a842257ed2add5134.dcm: [Errno 2] No such file or directory: 'D:/Digital_Pathology_Edge/data/raw/train/c91a842257ed2add5134.dcm'
Grouping annotations for add0a9bbc53d1d9bac4c.dcm...
  -> Processing 5518 patches...


d:\Digital_Pathology_Edge\src\patch_extraction.py:72: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  class_id = int(class_row['agreedClass'])


Grouping annotations for 96274538c93980aad8d6.dcm...
  -> Processing 7445 patches...
Error loading D:/Digital_Pathology_Edge/data/raw/train/f26e9fcef24609b988be.dcm: [Errno 2] No such file or directory: 'D:/Digital_Pathology_Edge/data/raw/train/f26e9fcef24609b988be.dcm'
Grouping annotations for 8c9f9618fcaca747b7c3.dcm...
  -> Processing 4053 patches...
Grouping annotations for be10fa37ad6e88e1f406.dcm...
  -> Processing 542 patches...
Error loading D:/Digital_Pathology_Edge/data/raw/train/dd4246ab756f6479c841.dcm: [Errno 2] No such file or directory: 'D:/Digital_Pathology_Edge/data/raw/train/dd4246ab756f6479c841.dcm'
Grouping annotations for 1018715d369dd0df2fc0.dcm...
  -> Processing 11221 patches...
Grouping annotations for 552c51bfb88fd3e65ffe.dcm...
  -> Processing 1537 patches...
Grouping annotations for c86cd41f96331adf3856.dcm...
  -> Processing 1199 patches...
Error loading D:/Digital_Pathology_Edge/data/raw/train/f3741e764d39ccc4d114.dcm: [Errno 2] No such file or directory: 

In [6]:
from coco_converter import patches_to_coco
os.makedirs(os.path.dirname(r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\train.json"), exist_ok=True)

train_dir = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\patches\train"

patches_to_coco(train_dir, r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\train.json")


COCO JSON saved → D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\train.json


In [ ]:
import os
import json
import shutil
from sklearn.model_selection import train_test_split

PATCH_DIR = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\patches\train"      # where ALL patches currently are
OUT_TRAIN = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\patches\train_split"
OUT_VAL   = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\patches\val_split"

COCO_IN   = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\train.json"
COCO_TRAIN_OUT = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\train_split.json"
COCO_VAL_OUT   = r"D:\Digital_Pathology_Edge\Mitosis_Detection\data\coco\val_split.json"

with open(COCO_IN, "r") as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]

train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

train_ids = {img["id"] for img in train_imgs}
val_ids   = {img["id"] for img in val_imgs}

train_ann = [ann for ann in annotations if ann["image_id"] in train_ids]
val_ann   = [ann for ann in annotations if ann["image_id"] in val_ids]

os.makedirs(OUT_TRAIN, exist_ok=True)
os.makedirs(OUT_VAL, exist_ok=True)

# Copy PNG + JSON pairs based on split
def copy_pair(img_list, out_dir):
    for img in img_list:
        fname = img["file_name"]    
        base  = os.path.splitext(fname)[0]

        src_png = os.path.join(PATCH_DIR, base + ".png")
        src_json = os.path.join(PATCH_DIR, base + ".json")

        dst_png = os.path.join(out_dir, base + ".png")
        dst_json = os.path.join(out_dir, base + ".json")

        if os.path.exists(src_png):
            shutil.copy(src_png, dst_png)
        if os.path.exists(src_json):
            shutil.copy(src_json, dst_json)

print("Copying TRAIN patches...")
copy_pair(train_imgs, OUT_TRAIN)

print("Copying VAL patches...")
copy_pair(val_imgs, OUT_VAL)

train_coco = {
    "images": train_imgs,
    "annotations": train_ann,
    "categories": coco["categories"]
}
val_coco = {
    "images": val_imgs,
    "annotations": val_ann,
    "categories": coco["categories"]
}

with open(COCO_TRAIN_OUT, "w") as f:
    json.dump(train_coco, f, indent=2)

with open(COCO_VAL_OUT, "w") as f:
    json.dump(val_coco, f, indent=2)

print("Done — train/val split created.")


Copying TRAIN patches...
Copying VAL patches...
Done — train/val split created.
